In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
# # Uncomment if you want to temporarily disable logger
# logger = logging.getLogger()
# logger.disabled = True

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    get_response_synthesizer,
)
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [5]:
os.listdir('./data/wiki')

['Algarve - Wikitravel.pdf',
 'Lisbon - Wikitravel.pdf',
 'Portugal - Wikitravel.pdf']

In [6]:
wiki_titles = os.listdir('./data/wiki')

In [7]:
# Load all wiki documents
city_docs = []
for wiki_title in wiki_titles:
    docs = SimpleDirectoryReader(input_files=[f"./data/wiki/{wiki_title}"]).load_data()
    docs[0].doc_id = wiki_title
    city_docs.extend(docs)

In [8]:
# # LLM Predictor (gpt-3.5-turbo)
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo-16k")
service_context = ServiceContext.from_defaults(llm=chatgpt, chunk_size=1024)

In [9]:
# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    city_docs,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
)

LLM is explicitly disabled. Using MockLLM.
current doc id: Algarve - Wikitravel.pdf
INFO:llama_index.indices.document_summary.base:> Generated summary for doc Algarve - Wikitravel.pdf: Context information from multiple sources is below.
---------------------
page_label: 1
file_name: Algarve - Wikitravel.pdf

Algarve
Earth > Europe > Iberia > Portugal > Southern Portugal > Algarve
The Algarve is the southernmost region of Portugal, on the coast of the Atlantic
Ocean.
Barlavento (Lagos, Silves, Portimão, Lagoa, Albufeira areas)
Serra Algarvia (Monchique, Caldeirão)
Sotavento (Faro, Loulé, São Brás de Alportel, Olhão, Tavira, Vila Real de Santo
António areas)
Faro — the regional capital
Albufeira
Lagos
Sagres
Monchique
Paderne
Portimão
Silves — first capital of the Algarve which has a Moorish castle
Tavira — a city near the Ria Formosa (lagoon).
Castro Marim
Quarteira
The Algarve is Portugal's most popular holiday destination due to the clean
beaches (approximately 200 km of them), the co

In [10]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [11]:
response = query_engine.query("Where to go in Algarve?")

In [12]:
print(response)

Faro, Albufeira, Lagos, Sagres, Monchique, Paderne, Portimão, Silves, Tavira, and Castro Marim are some popular destinations in the Algarve region of Portugal.


In [13]:
chat_engine = doc_summary_index.as_chat_engine(
    chat_mode='openai',
    verbose=False, 
    system_prompt = "You should answer in Czech"
)

In [14]:
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  What is the weather like in Faro in winter?


Assistant: V Faru panuje teplé klima s mírnými zimami. V zimních měsících jsou průměrné teploty během dne kolem 15°C a v noci kolem 9°C. Sníh a mráz jsou v Faru velmi vzácné. Doporučuje se přinést si navíc teplejší oblečení nebo deštník, protože město leží blízko oceánu a může zde být větrno a rychle se měnící počasí.



Human:  exit
